# Обучение RVector

In [1]:
import torch
from vocotrack import RVector
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from rvector_module import RVectorModule
from utils import PROCESSED_DATA_PATH
from data_module import DataModule
from lightning.pytorch import Trainer

device = 'cuda' if torch.cuda.is_available() else 'cpu'

%load_ext autoreload
%autoreload 2

print(f'[The device is {device}]')

[The device is cpu]


/Users/blnkoff/PythonProjects/vocotrack/.venv/lib/python3.12/site-packages/IPython/extensions/autoreload.py:215: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if not hasattr(module, "__file__") or module.__file__ is None:


In [2]:
config = {
    "res_net": {
        "stages": [
            {"in_channels": 32,  "num_channels": 32,  "blocks": 3, "stride": 1},
            {"in_channels": 32,  "num_channels": 64,  "blocks": 4, "stride": 2},
            {"in_channels": 64,  "num_channels": 128, "blocks": 6, "stride": 2},
            {"in_channels": 128, "num_channels": 256, "blocks": 3, "stride": 2},
        ]
    }
}


In [3]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/",             
    filename="rvector",  
    monitor="val_loss",                 
    mode="min",                        
    save_top_k=1,                      
    save_last=False,                   
    save_weights_only=True,            
)
early_stop = EarlyStopping(monitor="val_loss", patience=10, mode="min")

data_module = DataModule(PROCESSED_DATA_PATH)
module = RVectorModule(config)
trainer = Trainer(max_epochs=100, callbacks=[checkpoint_callback, early_stop])
trainer.fit(module, datamodule=data_module)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/blnkoff/PythonProjects/vocotrack/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
/Users/blnkoff/PythonProjects/vocotrack/.venv/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /Users/blnkoff/PythonProjects/vocotrack/notebooks/deep_learning/checkpoints exists and is not empty.

  | Name      | Type             | Params | Mode 
-------------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/blnkoff/PythonProjects/vocotrack/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:420: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.
/Users/blnkoff/PythonProjects/vocotrack/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:420: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [5]:
checkpoint_callback.best_model_score

tensor(0.3758, device='mps:0')